<a href="https://colab.research.google.com/github/a6iyyu/MachineLearning_3F_23/blob/main/JS6/P6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Praktikum 6**

Lakukan percobaan penggunaan ANNOY, FAISS, dan HNSWLIB pada dataset sekunder berukuran besar (Micro Spotify) pada link berikut: https://www.kaggle.com/datasets/bwandowando/spotify-songs-with-attributes-and-lyrics/data. Download data dan load CSV filenya (pilih dataset yg pertama dari dua dataset). pilih hanya fitur numerik saja, dan lakukan normalisasi menggunakan StandardScaler. Lakukan pencarian track terdekat dan bandingkan hasilnya.

In [1]:
!pip install pandas numpy scikit-learn annoy hnswlib faiss-cpu opendatasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 21.8 MB/s eta 0:00:00
  Created wheel for annoy: filename=annoy-1.17.3-cp312-cp312-linux_x86_64.whl size=551811 sha256=d77a961f459243f5bb0d482f13a84329101682dec889027d0f7c6f5907c7a961
  Stored in directory: /root/.cache/pip/wheels/db/b9/53/a3b2d1fe1743abadddec6aa541294b24fdbc39d7800bc57311
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp312-cp312-linux_x86_64.whl size=2528144 sha256=4bf1e7707782067e7cd50482b36503a9a9120201cc5a835d4dba3fac696051d8
  Stored in directory: /root/.cache/pip/wheels/ac/39/b3/cbd7f9cbb76501d2d5fbc84956e70d0b94e788aac87bda465e
Successfully built annoy hnswlib


In [2]:
import pandas as pd
import numpy as np
import time
import faiss
from annoy import AnnoyIndex
import hnswlib
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import opendatasets as od
import os

In [ ]:
dataset_url = 'https://www.kaggle.com/datasets/bwandowando/spotify-songs-with-attributes-and-lyrics'
dataset_dir = 'spotify-songs-with-attributes-and-lyrics'

# Path ke file CSV yang akan digunakan (dataset pertama)
csv_path = os.path.join(dataset_dir, 'songs_with_attributes_and_lyrics.csv')

# Cek apakah dataset sudah ada, jika tidak, unduh
if not os.path.exists(csv_path):
    print("File dataset tidak ditemukan. Mengunduh dari Kaggle...")
    od.download(dataset_url)
else:
    print("Dataset sudah tersedia di direktori.")

print(f"Membaca file dari: {csv_path}")

try:
    df = pd.read_csv(csv_path)
    print("Dataset berhasil dimuat.")
    print(f"Jumlah data: {len(df)} lagu")
except FileNotFoundError:
    print(f"Error: File tidak ditemukan di path: {csv_path}")
    print("Pastikan proses download berhasil dan path file sudah benar.")

File dataset tidak ditemukan. Mengunduh dari Kaggle...
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

In [ ]:
try:
    df = pd.read_csv(csv_path)
    print("Dataset berhasil dimuat.")
    print(f"Jumlah data: {len(df)} lagu")
except FileNotFoundError:
    print(f"Error: File tidak ditemukan di path: {csv_path}")
    print("Pastikan proses download berhasil dan path file sudah benar.")

In [ ]:
# Pilih hanya fitur numerik yang relevan untuk perbandingan kemiripan lagu
features = ['danceability', 'energy', 'loudness', 'speechiness',
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
X = df[features].values.astype('float32') # FAISS bekerja lebih baik dengan float32

In [ ]:
# -------------------------------
# 2. Standarisasi Fitur
# -------------------------------
# Scaling penting agar fitur dengan rentang nilai besar (misal: tempo)
# tidak mendominasi fitur dengan rentang kecil (misal: danceability).
print("\nMelakukan standarisasi fitur...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Standarisasi selesai.")

# Jumlah nearest neighbors yang akan dicari
k = 10

print("\nMemulai perbandingan metode Nearest Neighbor Search...")
print("-" * 50)

In [ ]:
# -------------------------------
# Metode 1: Exact Nearest Neighbor (Brute-force) sebagai Baseline
# -------------------------------
start_time = time.time()
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(X_scaled)
# Mencari tetangga terdekat untuk semua titik data
dist_exact, idx_exact = nn.kneighbors(X_scaled)
time_exact = time.time() - start_time
print(f"Exact NN (Brute-force) selesai dalam: {time_exact:.4f} detik")

In [ ]:
# -------------------------------
# Metode 2: Annoy (Approximate Nearest Neighbors Oh Yeah)
# -------------------------------
start_time = time.time()
num_features = X_scaled.shape[1]
index_annoy = AnnoyIndex(num_features, 'euclidean')
for i, vec in enumerate(X_scaled):
    index_annoy.add_item(i, vec)

# Membangun index. Angka 10 adalah jumlah pohon (n_trees).
# Semakin banyak pohon, semakin akurat hasilnya, tapi semakin lama proses build-nya.
index_annoy.build(10)

# Mencari tetangga
idx_annoy = []
for vec in X_scaled:
    idx_annoy.append(index_annoy.get_nns_by_vector(vec, k))
time_annoy = time.time() - start_time
print(f"Annoy selesai dalam:                   {time_annoy:.4f} detik")

In [ ]:
# -------------------------------
# Metode 3: HNSW (Hierarchical Navigable Small World)
# -------------------------------
start_time = time.time()
dim = X_scaled.shape[1]
num_elements = X_scaled.shape[0]

# Deklarasi index
p_hnsw = hnswlib.Index(space='l2', dim=dim) # l2 space = euclidean distance

# Inisialisasi index. Parameter M dan ef_construction mempengaruhi akurasi vs kecepatan build.
p_hnsw.init_index(max_elements=num_elements, ef_construction=200, M=16)

# Menambahkan data ke index
p_hnsw.add_items(X_scaled)

# Mengatur parameter pencarian (ef). Semakin tinggi, semakin akurat tapi lebih lambat.
p_hnsw.set_ef(200)

# Melakukan pencarian
idx_hnsw, dist_hnsw = p_hnsw.knn_query(X_scaled, k=k)
time_hnsw = time.time() - start_time
print(f"HNSW selesai dalam:                     {time_hnsw:.4f} detik")

In [ ]:
# -------------------------------
# Metode 4: FAISS (Facebook AI Similarity Search) - IVF
# -------------------------------
start_time = time.time()
dim = X_scaled.shape[1]
nlist = 100  # Jumlah sel/partisi. Aturan umum: akar kuadrat dari jumlah data.

# Quantizer untuk mempartisi data
quantizer = faiss.IndexFlatL2(dim)
# Index IVF (Inverted File) yang menggunakan quantizer
index_faiss = faiss.IndexIVFFlat(quantizer, dim, nlist, faiss.METRIC_L2)

# Training index untuk mempelajari distribusi data dan membuat partisi
index_faiss.train(X_scaled)
# Menambahkan data ke index
index_faiss.add(X_scaled)

# nprobe: Berapa banyak partisi terdekat yang akan dicari.
# Semakin tinggi, semakin akurat tapi semakin lambat.
index_faiss.nprobe = 10

# Melakukan pencarian
dist_faiss, idx_faiss = index_faiss.search(X_scaled, k)
time_faiss = time.time() - start_time
print(f"FAISS IVF selesai dalam:                {time_faiss:.4f} detik")
print("-" * 50)

In [ ]:
# -------------------------------
# 5. Perbandingan Hasil
# -------------------------------
# Kita akan membandingkan hasil pencarian untuk lagu pertama dalam dataset.
# Lagu pertama dianggap sebagai dirinya sendiri, jadi kita lihat 5 tetangga setelahnya.
query_song_index = 0
num_neighbors_to_show = 6 # Termasuk dirinya sendiri

print(f"\nTop-{num_neighbors_to_show-1} neighbors untuk lagu pertama (index {query_song_index}):")
print(f"Lagu Asli: '{df.iloc[query_song_index]}' oleh {df.iloc[query_song_index]['track_artist']}")
print("-" * 50)

In [ ]:
# Fungsi untuk menghitung recall@k
def calculate_recall(ground_truth, approx_results):
    correct_predictions = 0
    total_queries = len(ground_truth)
    for i in range(total_queries):
        # Mengabaikan item pertama karena itu adalah item itu sendiri
        true_neighbors = set(ground_truth[i][1:])
        approx_neighbors = set(approx_results[i][1:])
        correct_predictions += len(true_neighbors.intersection(approx_neighbors))

    # Total tetangga yang seharusnya ditemukan (k-1 per query)
    total_neighbors = total_queries * (k - 1)
    return correct_predictions / total_neighbors if total_neighbors > 0 else 0

In [ ]:
# Tampilkan hasil dan hitung akurasi (recall)
recall_annoy = calculate_recall(idx_exact, idx_annoy)
recall_hnsw = calculate_recall(idx_exact, idx_hnsw.tolist())
recall_faiss = calculate_recall(idx_exact, idx_faiss.tolist())

print(f"Baseline (Exact): {idx_exact[query_song_index][:num_neighbors_to_show]}")
print(f"Annoy:            {np.array(idx_annoy)[query_song_index][:num_neighbors_to_show]} (Recall: {recall_annoy:.4f})")
print(f"HNSW:             {idx_hnsw[query_song_index][:num_neighbors_to_show]} (Recall: {recall_hnsw:.4f})")
print(f"FAISS:            {idx_faiss[query_song_index][:num_neighbors_to_show]} (Recall: {recall_faiss:.4f})")